# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [3]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

W0814 09:20:20.575450 140371677804416 deprecation.py:323] From <ipython-input-3-055975cd90d7>:10: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0814 09:20:20.584722 140371677804416 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6417222023
GPU (s):
2.28791713715
GPU speedup over CPU: 1x


#  Let's just try out inference using a pre-trained model

In [4]:
%cd /content

/content


In [0]:
!mkdir Tensorflow_object_detector

In [6]:
%cd /content/Tensorflow_object_detector

/content/Tensorflow_object_detector


## Clone the models repo

In [7]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 29666, done.
remote: Total 29666 (delta 0), reused 0 (delta 0), pack-reused 29666
Receiving objects: 100% (29666/29666), 510.15 MiB | 26.79 MiB/s, done.
Resolving deltas: 100% (18501/18501), done.
Checking out files: 100% (3062/3062), done.


In [8]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/Tensorflow_object_detector/models/research

Cloning into 'cocoapi'...
remote: Enumerating objects: 959, done.
remote: Total 959 (delta 0), reused 0 (delta 0), pack-reused 959
Receiving objects: 100% (959/959), 11.70 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (569/569), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python2.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Tensorflow_object_detector/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-2.7
creating build/temp.linux-x86_64-2.7/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fno-strict-aliasing -Wdate-time -D_FORTIFY_SOURCE=2 -g -fdebug-prefix-map=/build/python2.7-

In [0]:
!mkdir workspace


In [10]:
%cd /content/Tensorflow_object_detector/workspace
!mkdir training_OD

/content/Tensorflow_object_detector/workspace


In [11]:
%cd /content/Tensorflow_object_detector/workspace/training_OD
!mkdir annotations
#!mkdir images
!mkdir pre-trained_model
!mkdir training
!mkdir scripts

/content/Tensorflow_object_detector/workspace/training_OD


## Build object detection module

In [12]:
%cd /content/Tensorflow_object_detector/models/research

/content/Tensorflow_object_detector/models/research


In [13]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.16-2~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 4 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 http

In [14]:
%cd /content/Tensorflow_object_detector/models/research

/content/Tensorflow_object_detector/models/research


In [0]:
# in research folder 
!protoc object_detection/protos/*.proto --python_out=.

## Basic imports

In [16]:
%cd /content/Tensorflow_object_detector/models/research/object_detection

/content/Tensorflow_object_detector/models/research/object_detection


In [0]:
from distutils.version import StrictVersion
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import shutil

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import shutil

# This is needed we are in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

## Object Detection imports

In [18]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# ignore the warning

../object_detection/utils/visualization_utils.py:29: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/

## Loading

In [0]:
import shutil

In [20]:
%cd /content/Tensorflow_object_detector/workspace/training_OD/images

[Errno 2] No such file or directory: '/content/Tensorflow_object_detector/workspace/training_OD/images'
/content/Tensorflow_object_detector/models/research/object_detection


In [0]:
shutil.copytree("/content/drive/My Drive/RFCN/jpg","/content/Tensorflow_object_detector/workspace/training_OD/images/jpg")


In [0]:
shutil.copy("/content/drive/My Drive/RFCN/annotation.csv","/content/Tensorflow_object_detector/workspace/training_OD/annotations")

## Image Preprocessing

### Image reseizing

In [0]:
'''
directory ="/content/Tensorflow_object_detector/workspace/training_OD/images/jpg"

for file_name in os.listdir(directory):
  print("Processing %s" % file_name)
  image = Image.open(os.path.join(directory, file_name))

  
  new_dimensions = (224, 224)
  output = image.resize(new_dimensions, Image.ANTIALIAS)
  folder = '/content/Tensorflow_object_detector/workspace/training_OD/images/reseizedImages'
  if not os.path.exists(folder):
    os.makedirs(folder)
  output_file_name = os.path.join(folder, file_name)
  #output.save(output_file_name, "JPEG", quality = 95)
  output.save(output_file_name)

print("All done")

'''

## Converting from Grayscale to RGB

In [24]:

RESEIZE ='/content/Tensorflow_object_detector/workspace/training_OD/images/jpg'
for file_name in os.listdir(RESEIZE):
  print("Processing %s" % file_name)
  
  image = Image.open(os.path.join(RESEIZE, file_name))
  image.show()
  rgb_image = image.convert('RGB')
  rgb_image.show()
  print(rgb_image.mode)

  dir = '/content/Tensorflow_object_detector/workspace/training_OD/images/RGBImages'
  if not os.path.exists(dir):
       os.makedirs(dir)
  #output_file_name = os.path.join(dir, "rgb_" + file_name)
  output_file_name = os.path.join(dir,file_name)
  rgb_image.save(output_file_name)
print("All done")



Processing ti10161457.jpg
RGB
Processing 1156_004.jpg
RGB
Processing ti15269402.jpg
RGB
Processing 6546_016.jpg
RGB
Processing 5830_158.jpg
RGB
Processing 5170_020.jpg
RGB
Processing 1675_199.jpg
RGB
Processing 9551_062.jpg
RGB
Processing 0000226470.jpg
RGB
Processing 9526_017.jpg
RGB
Processing 9527_018.jpg
RGB
Processing 1558_040.jpg
RGB
Processing 0212_175.jpg
RGB
Processing 6790_011.jpg
RGB
Processing 9510_041.jpg
RGB
Processing 9566_031.jpg
RGB
Processing 0651_013.jpg
RGB
Processing 5001_024.jpg
RGB
Processing 1796_095.jpg
RGB
Processing 5830_144.jpg
RGB
Processing 5830_462.jpg
RGB
Processing 5347_192.jpg
RGB
Processing 5373_008.jpg
RGB
Processing 5366_142.jpg
RGB
Processing 1556_004.jpg
RGB
Processing ti10161630.jpg
RGB
Processing 5182_078.jpg
RGB
Processing 9503_001.jpg
RGB
Processing 0000308007.jpg
RGB
Processing 5856_026.jpg
RGB
Processing ti14940067.jpg
RGB
Processing 5622_013.jpg
RGB
Processing 5140_040.jpg
RGB
Processing ti11492415_2417.jpg
RGB
Processing 9502_055.jpg
RGB
P

In [0]:
from matplotlib import pyplot as plt
#image = Image.open(os.path.join('/content/Tensorflow_object_detector/workspace/training_OD/images/RGBImages,'0000173065.jpg'))
#print(image.mode)
#print(image.size)
#image_np = load_image_into_numpy_array(image)
#plt.imshow(image_np)


## Splitting Dataset into train and test

In [0]:
    import pandas as pd

    import numpy as np

    #df = pd.read_csv('/content/drive/My Drive/Tensorflow_object_detector/workspace/training_OD/annotations/mydataset.csv',error_bad_lines=False)
    df = pd.read_csv('/content/Tensorflow_object_detector/workspace/training_OD/annotations/annotation.csv',error_bad_lines=False)

    df['split'] = np.random.randn(df.shape[0], 1)

    msk = np.random.rand(len(df)) <= 0.8

    train = df[msk]

    test = df[~msk]

    train.to_csv('/content/Tensorflow_object_detector/workspace/training_OD/annotations/train.csv', index=None)
    test.to_csv('/content/Tensorflow_object_detector/workspace/training_OD/annotations/test.csv', index=None)

## Label map

# Creating TFRecords

In [27]:
%cd /content/Tensorflow_object_detector/workspace/training_OD/scripts
!mkdir preprocessing

/content/Tensorflow_object_detector/workspace/training_OD/scripts


In [28]:
%cd /content/Tensorflow_object_detector/models/research
!python setup.py build
!python setup.py install

/content/Tensorflow_object_detector/models/research
running build
running build_py
creating build
creating build/lib.linux-x86_64-2.7
creating build/lib.linux-x86_64-2.7/object_detection
copying object_detection/export_inference_graph.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/exporter.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/eval_util_test.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/model_lib_test.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/inputs_test.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/model_lib_v2_test.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/model_lib_v2.py -> build/lib.linux-x86_64-2.7/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib.linux-x86_

In [1]:
%cd /content/Tensorflow_object_detector/workspace/training_OD

/content/Tensorflow_object_detector/workspace/training_OD


In [5]:
import shutil
shutil.copy("/content/drive/My Drive/RFCN/generate_TFRecord.py","/content/Tensorflow_object_detector/workspace/training_OD/scripts/preprocessing")

'/content/Tensorflow_object_detector/workspace/training_OD/scripts/preprocessing/generate_TFRecord.py'

In [6]:
#!python scripts/preprocessing/generate_TFRecord.py --csv_input=annotations/train.csv --output_path=annotations/train.record --image_dir=
!python scripts/preprocessing/generate_TFRecord.py --csv_input=annotations/train.csv --output_path=annotations/train.record 

Traceback (most recent call last):
  File "scripts/preprocessing/generate_TFRecord.py", line 19, in <module>
    from object_detection.utils import dataset_util
ModuleNotFoundError: No module named 'object_detection'


In [12]:
!python scripts/preprocessing/generate_TFRecord.py --csv_input=annotations/test.csv --output_path=annotations/eval.record 

W0814 09:53:13.855981 140385074640768 deprecation_wrapper.py:119] From scripts/preprocessing/generate_TFRecord.py:104: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0814 09:53:13.856658 140385074640768 deprecation_wrapper.py:119] From scripts/preprocessing/generate_TFRecord.py:90: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0814 09:53:13.948018 140385074640768 deprecation_wrapper.py:119] From scripts/preprocessing/generate_TFRecord.py:48: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/Tensorflow_object_detector/workspace/training_OD/annotations/eval.record


## Loading the pretrained model

In [33]:
%cd /content/Tensorflow_object_detector/workspace/training_OD/pre-trained_model

/content/Tensorflow_object_detector/workspace/training_OD/pre-trained_model


In [34]:
!wget http://download.tensorflow.org/models/object_detection/rfcn_resnet101_coco_2018_01_28.tar.gz

--2019-08-14 09:26:07--  http://download.tensorflow.org/models/object_detection/rfcn_resnet101_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.71.128, 2a00:1450:400c:c02::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.71.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 652955004 (623M) [application/x-tar]
Saving to: ‘rfcn_resnet101_coco_2018_01_28.tar.gz’

rfcn_resnet101_coco 100%[===================>] 622.71M   184MB/s    in 3.4s    

2019-08-14 09:26:10 (184 MB/s) - ‘rfcn_resnet101_coco_2018_01_28.tar.gz’ saved [652955004/652955004]



In [35]:
!tar -C /content/Tensorflow_object_detector/workspace/training_OD/pre-trained_model  -xzvf /content/Tensorflow_object_detector/workspace/training_OD/pre-trained_model/rfcn_resnet101_coco_2018_01_28.tar.gz

rfcn_resnet101_coco_2018_01_28/
rfcn_resnet101_coco_2018_01_28/model.ckpt.index
rfcn_resnet101_coco_2018_01_28/checkpoint
rfcn_resnet101_coco_2018_01_28/pipeline.config
rfcn_resnet101_coco_2018_01_28/model.ckpt.data-00000-of-00001
rfcn_resnet101_coco_2018_01_28/model.ckpt.meta
rfcn_resnet101_coco_2018_01_28/saved_model/
rfcn_resnet101_coco_2018_01_28/saved_model/saved_model.pb
rfcn_resnet101_coco_2018_01_28/saved_model/variables/
rfcn_resnet101_coco_2018_01_28/frozen_inference_graph.pb


## Training

In [0]:
shutil.copy("/content/drive/My Drive/RFCN/rfcn_resnet101_coco.config","/content/Tensorflow_object_detector/workspace/training_OD/training")

In [0]:
shutil.copy("/content/drive/My Drive/RFCN/train.py","/content/Tensorflow_object_detector/workspace/training_OD")

In [0]:
shutil.copy("/content/drive/My Drive/RFCN/dataset_label_map.pbtxt","/content/Tensorflow_object_detector/workspace/training_OD/annotations")

In [0]:
#fixes nets import error 
import os
os.environ['PYTHONPATH'] += ':/content/Tensorflow_object_detector/models/research/:/content/Tensorflow_object_detector/models/research/slim/'

In [14]:
%cd /content/Tensorflow_object_detector/workspace/training_OD

/content/Tensorflow_object_detector/workspace/training_OD


In [15]:
!python /content/Tensorflow_object_detector/workspace/training_OD/train.py --logtostderr --train_dir=training --pipeline_config_path=training/rfcn_resnet101_coco.config

W0814 09:53:34.129182 140244373849984 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0814 09:53:34.263947 140244373849984 deprecation_wrapper.py:119] From /content/Tensorflow_object_detector/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0814 09:53:34.272061 140244373849984 deprecation_wrapper.py:119] From /content/Tensorflow_object_detector/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0814 09:53:34.281439 140244373849984 deprecation_wrapper.py:119] From /content/Tensorflow_object_

## Training Evaluation

In [0]:
shutil.copy("/content/drive/My Drive/RFCN/eval.py","/content/Tensorflow_object_detector/workspace/training_OD")

In [0]:
%cd /content/Tensorflow_object_detector/workspace/training_OD

In [0]:
!python /content/Tensorflow_object_detector/workspace/training_OD/eval.py --logtostderr --pipeline_config_path=training/rfcn_resnet101_coco.config --checkpoint_dir=training/ --eval_dir=training/

## Tensorboard

In [0]:
%cd /content/Tensorflow_object_detector/workspace/training_OD

In [0]:
tensorboard --logdir=training